In [1]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Build DB URL
DATABASE_URL = os.getenv("DATABASE_URL") or \
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@" \
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

def test_connection():
    try:
        engine = create_engine(DATABASE_URL)
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1")).scalar()
            if result == 1:
                print("✔ Connection Successful! DB is reachable.")
    except Exception as e:
        print(f"❌ Connection Failed:\n{e}")

if __name__ == "__main__":
    test_connection()


✔ Connection Successful! DB is reachable.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import re

# Load Environment Variables
load_dotenv()

# Get DB URL
DATABASE_URL = os.getenv("DATABASE_URL") or \
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@" \
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

# File path
file_path = r"C:\Users\trilo\OneDrive\Desktop\Report_Transactions\transactions.xlsx"

# Read Excel
df = pd.read_excel(file_path)
df['Category'] = df['Category'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x).strip())
df['Period'] = df['Period'].apply(lambda x: re.sub(r'\d{2}:\d{2}:\d{2}', '', str(x)).strip())
df['Period'] = pd.to_datetime(df['Period'], format='%Y-%m-%d')


# Upload to Render DB
engine = create_engine(DATABASE_URL)
df.to_sql("finance_data", engine, if_exists="append", index=False)

print("✔ Upload completed successfully!")


✔ Upload completed successfully!


In [5]:
df.tail()

,Period,Accounts,Category,Amount,Month,% Row,Running Total
574,2025-11-03 21:28:41,Upi,🍜 Food,21.0,November,0.000065,313638.96
575,2025-11-01 20:35:34,Upi,Recharge,588.0,November,0.001825,314226.96
576,2025-11-01 20:35:19,Upi,Rent,6149.0,November,0.019081,320375.96
577,2025-11-01 20:34:55,Upi,🚖 Transport,1400.0,November,0.004344,321775.96
578,2025-11-01 16:18:40,Upi,🚖 Transport,478.0,November,0.001483,322253.96


In [9]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL") or \
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@" \
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

engine = create_engine(DATABASE_URL)

print("\n🔍 Checking & Fixing finance_data schema...\n")

# ===========================================================
# SAFE DDL EXECUTOR (Fixes SQLAlchemy engine.execute removal)
# ===========================================================
def run(sql):
    try:
        with engine.connect() as conn:
            conn.execute(text(sql))
            conn.commit()
    except Exception as e:
        print(f"⚠ {e}")

# ===========================================================
# STEP 1 — RENAME wrong columns if exist
# ===========================================================
rename_map = {
    "Period": "period",
    "Accounts": "accounts",
    "Category": "category",
    "Amount": "amount",
    "Month": "month",
    "% Row": "percent_row",
    "Running Total": "running_total"
}

for old,new in rename_map.items():
    run(f'ALTER TABLE finance_data RENAME COLUMN "{old}" TO {new};')
    print(f"✔ Checked rename: {old} → {new}")

# ===========================================================
# STEP 2 — ADD missing columns with correct data types
# ===========================================================
required_columns = {
    "period": "DATE",
    "accounts": "TEXT",
    "category": "TEXT",
    "amount": "NUMERIC",
    "income": "NUMERIC",
    "month": "TEXT",
    "percent_row": "NUMERIC",
    "running_total": "NUMERIC"
}

for col, dtype in required_columns.items():
    run(f"ALTER TABLE finance_data ADD COLUMN IF NOT EXISTS {col} {dtype};")
    print(f"✔ Ensured column exists: {col} ({dtype})")

print("\n🎉 DATABASE UPDATED SUCCESSFULLY — Now your Streamlit app will INSERT without error.\n")



🔍 Checking & Fixing finance_data schema...

✔ Checked rename: Period → period
✔ Checked rename: Accounts → accounts
✔ Checked rename: Category → category
✔ Checked rename: Amount → amount
✔ Checked rename: Month → month
✔ Checked rename: % Row → percent_row
✔ Checked rename: Running Total → running_total
✔ Ensured column exists: period (DATE)
✔ Ensured column exists: accounts (TEXT)
✔ Ensured column exists: category (TEXT)
✔ Ensured column exists: amount (NUMERIC)
✔ Ensured column exists: income (NUMERIC)
✔ Ensured column exists: month (TEXT)
✔ Ensured column exists: percent_row (NUMERIC)
✔ Ensured column exists: running_total (NUMERIC)

🎉 DATABASE UPDATED SUCCESSFULLY — Now your Streamlit app will INSERT without error.

